In [ ]:


import pandas as pd # pyright: ignore[reportMissingModuleSource]
import numpy as np # pyright: ignore[reportMissingImports]
import matplotlib.pyplot as plt # pyright: ignore[reportMissingModuleSource]
from sklearn.impute import SimpleImputer # pyright: ignore[reportMissingModuleSource]
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler # pyright: ignore[reportMissingModuleSource]
from sklearn.ensemble import RandomForestRegressor # type: ignore
from sklearn.model_selection import train_test_split # pyright: ignore[reportMissingModuleSource]
from sklearn.decomposition import PCA # pyright: ignore[reportMissingModuleSource]
from sklearn.cluster import KMeans # pyright: ignore[reportMissingModuleSource]
from sklearn.feature_selection import mutual_info_regression # pyright: ignore[reportMissingModuleSource]
from scipy.stats import ttest_ind, f_oneway, pearsonr # pyright: ignore[reportMissingImports]
import math

In [5]:
fp = "dataset.csv"
df = pd.read_csv(fp)

In [6]:
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      18368 non-null  int64  
 1   SystemCodeNumber        18368 non-null  object 
 2   Capacity                18368 non-null  int64  
 3   Latitude                18368 non-null  float64
 4   Longitude               18368 non-null  float64
 5   Occupancy               18368 non-null  int64  
 6   VehicleType             18368 non-null  object 
 7   TrafficConditionNearby  18368 non-null  object 
 8   QueueLength             18368 non-null  int64  
 9   IsSpecialDay            18368 non-null  int64  
 10  LastUpdatedDate         18368 non-null  object 
 11  LastUpdatedTime         18368 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.7+ MB


In [8]:
df.shape

(18368, 12)

In [9]:
df.columns.tolist()

['ID',
 'SystemCodeNumber',
 'Capacity',
 'Latitude',
 'Longitude',
 'Occupancy',
 'VehicleType',
 'TrafficConditionNearby',
 'QueueLength',
 'IsSpecialDay',
 'LastUpdatedDate',
 'LastUpdatedTime']

In [10]:
missing = df.isna().sum().sort_values(ascending=False)
print(missing)


ID                        0
SystemCodeNumber          0
Capacity                  0
Latitude                  0
Longitude                 0
Occupancy                 0
VehicleType               0
TrafficConditionNearby    0
QueueLength               0
IsSpecialDay              0
LastUpdatedDate           0
LastUpdatedTime           0
dtype: int64


In [ ]:
df.columns = [c.strip().replace(" ", "_") for c in df.columns]

In [ ]:
if "LastUpdatedDate" in df.columns or "LastUpdatedTime" in df.columns:
    # create LastUpdatedTimestamp where possible
    date_col = "LastUpdatedDate" if "LastUpdatedDate" in df.columns else None
    time_col = "LastUpdatedTime" if "LastUpdatedTime" in df.columns else None
    if date_col and time_col:
        # combine safely
        def combine_datetime(row):
            d = row.get(date_col)
            t = row.get(time_col)
            if pd.isna(d) and pd.isna(t):
                return pd.NaT
            try:
                return pd.to_datetime(str(d) + " " + str(t), format='%d-%m-%Y %H:%M:%S', errors='coerce')
            except Exception:
                try:
                    return pd.to_datetime(d)
                except Exception:
                    return pd.NaT
        df["LastUpdatedTimestamp"] = df.apply(combine_datetime, axis=1)
    elif date_col:
        df["LastUpdatedTimestamp"] = pd.to_datetime(df[date_col], errors="coerce")
    else:
        df["LastUpdatedTimestamp"] = pd.to_datetime(df[time_col], errors="coerce")
else:
    df["LastUpdatedTimestamp"] = pd.NaT

print("\nParsed LastUpdatedTimestamp sample:")
df[["LastUpdatedDate","LastUpdatedTime","LastUpdatedTimestamp"]].head(10)


Parsed LastUpdatedTimestamp sample:


,LastUpdatedDate,LastUpdatedTime,LastUpdatedTimestamp
0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,04-10-2016,08:25:00,2016-10-04 08:25:00
2,04-10-2016,08:59:00,2016-10-04 08:59:00
3,04-10-2016,09:32:00,2016-10-04 09:32:00
4,04-10-2016,09:59:00,2016-10-04 09:59:00
5,04-10-2016,10:26:00,2016-10-04 10:26:00
6,04-10-2016,10:59:00,2016-10-04 10:59:00
7,04-10-2016,11:25:00,2016-10-04 11:25:00
8,04-10-2016,11:59:00,2016-10-04 11:59:00
9,04-10-2016,12:29:00,2016-10-04 12:29:00


In [ ]:
dup_count = df.duplicated().sum()
print("Duplicate rows:", dup_count)
if dup_count>0:
    df = df.drop_duplicates().reset_index(drop=True)
    print("Dropped duplicates. New shape:", df.shape)

Duplicate rows: 0


In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
non_numeric = [c for c in df.columns if c not in numeric_cols]
print("Numeric columns:", numeric_cols)
print("Other columns:", non_numeric)

Numeric columns: ['ID', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'QueueLength', 'IsSpecialDay']
Other columns: ['SystemCodeNumber', 'VehicleType', 'TrafficConditionNearby', 'LastUpdatedDate', 'LastUpdatedTime', 'LastUpdatedTimestamp']


In [ ]:
num_cols_for_impute = [c for c in numeric_cols if c not in ["ID","SystemCodeNumber"]]
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print("\nNumeric columns used for numeric imputation:", num_cols_for_impute)
for c in num_cols_for_impute:
    if df[c].isna().sum()>0:
        med = df[c].median()
        df[c] = df[c].fillna(med)
        print(f"Imputed numeric {c} with median {med} (filled {df[c].isna().sum()} remaining NAs)")

# Impute categorical with 'Unknown'
for c in non_numeric:
    if df[c].isna().sum()>0:
        df[c] = df[c].fillna("Unknown")


Numeric columns used for numeric imputation: ['Capacity', 'Latitude', 'Longitude', 'Occupancy', 'QueueLength', 'IsSpecialDay']


In [ ]:
print("\nAfter basic imputation missing counts:")
print(df.isna().sum().sort_values(ascending=False).head(20))


After basic imputation missing counts:
ID                        0
SystemCodeNumber          0
Capacity                  0
Latitude                  0
Longitude                 0
Occupancy                 0
VehicleType               0
TrafficConditionNearby    0
QueueLength               0
IsSpecialDay              0
LastUpdatedDate           0
LastUpdatedTime           0
LastUpdatedTimestamp      0
dtype: int64


In [ ]:
interesting_nums = [c for c in ["Price","Occupancy","Capacity","QueueLength","TrafficScore","VehicleScore"] if c in df.columns]
outlier_summary = {}
for c in interesting_nums:
    q1 = df[c].quantile(0.25)
    q3 = df[c].quantile(0.75)
    iqr = q3 - q1
    low = q1 - 1.5*iqr
    high = q3 + 1.5*iqr
    outlier_mask = (df[c] < low) | (df[c] > high)
    outlier_summary[c] = int(outlier_mask.sum())
    print(f"{c}: {outlier_summary[c]} suspected outliers (IQR method)")
    
# We won't drop outliers automatically but will flag them


Occupancy: 867 suspected outliers (IQR method)
Capacity: 0 suspected outliers (IQR method)
QueueLength: 59 suspected outliers (IQR method)


In [ ]:
for c in interesting_nums:
    df[f"{c}_outlier_flag"] = 0
    q1 = df[c].quantile(0.25)
    q3 = df[c].quantile(0.75)
    iqr = q3 - q1
    low = q1 - 1.5*iqr
    high = q3 + 1.5*iqr
    df.loc[(df[c] < low) | (df[c] > high), f"{c}_outlier_flag"] = 1

In [ ]:
if ("Occupancy" in df.columns) and ("Capacity" in df.columns):
    
    df["occupancy_ratio"] = df.apply(lambda r: r["Occupancy"]/r["Capacity"] if (pd.notna(r["Occupancy"]) and pd.notna(r["Capacity"]) and r["Capacity"]>0) else 0.0, axis=1)
    print("Created occupancy_ratio")

Created occupancy_ratio


In [ ]:
if "LastUpdatedTimestamp" in df.columns:
    df["hour"] = df["LastUpdatedTimestamp"].dt.hour.fillna(-1).astype(int)
    df["dayofweek"] = df["LastUpdatedTimestamp"].dt.dayofweek.fillna(-1).astype(int)
    df["is_weekend"] = df["dayofweek"].apply(lambda x: 1 if x in [5,6] else 0 if x>=0 else 0)

    df["hour_sin"] = df["hour"].apply(lambda h: math.sin(2*math.pi*h/24) if h>=0 else 0)
    df["hour_cos"] = df["hour"].apply(lambda h: math.cos(2*math.pi*h/24) if h>=0 else 0)
    print("Created hour, dayofweek, is_weekend, hour_sin, hour_cos")

Created hour, dayofweek, is_weekend, hour_sin, hour_cos


In [ ]:
cat_cols = []
for c in ["VehicleType","TrafficConditionNearby"]:
    if c in df.columns:
        cat_cols.append(c)
        df[c] = df[c].astype(str).str.strip().fillna("Unknown")

print("Categorical columns to encode:", cat_cols)

for c in cat_cols:
    le = LabelEncoder()
    try:
        df[f"{c}_label"] = le.fit_transform(df[c])
    except Exception:
        df[f"{c}_label"] = df[c].astype('category').cat.codes
for c in cat_cols:
    dummies = pd.get_dummies(df[c], prefix=c)
    
    if dummies.shape[1] <= 10:
        df = pd.concat([df, dummies], axis=1)

Categorical columns to encode: ['VehicleType', 'TrafficConditionNearby']


In [ ]:
if ("Latitude" in df.columns) and ("Longitude" in df.columns):
    coords = df[["Latitude","Longitude"]].dropna()
    # select k based on sqrt heuristic but capped
    k = min(max(3, int(math.sqrt(len(coords)))), 10)
    if len(coords) >= k:
        kmeans = KMeans(n_clusters=k, random_state=42)
        cluster_labels = kmeans.fit_predict(coords)
        # map back to df (careful about index)
        df.loc[coords.index, "zone_cluster"] = cluster_labels
        df["zone_cluster"] = df["zone_cluster"].fillna(-1).astype(int)
        print(f"Created zone_cluster via KMeans with k={k}")

Created zone_cluster via KMeans with k=10


In [ ]:
df.head(10)



,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,...,VehicleType_label,TrafficConditionNearby_label,VehicleType_bike,VehicleType_car,VehicleType_cycle,VehicleType_truck,TrafficConditionNearby_average,TrafficConditionNearby_high,TrafficConditionNearby_low,zone_cluster
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,...,1,2,False,True,False,False,False,False,True,3
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,...,1,2,False,True,False,False,False,False,True,3
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,...,1,2,False,True,False,False,False,False,True,3
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,...,1,2,False,True,False,False,False,False,True,3
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,...,0,2,True,False,False,False,False,False,True,3
5,5,BHMBCCMKT01,577,26.144536,91.736172,177,car,low,3,0,...,1,2,False,True,False,False,False,False,True,3
6,6,BHMBCCMKT01,577,26.144536,91.736172,219,truck,high,6,0,...,3,1,False,False,False,True,False,True,False,3
7,7,BHMBCCMKT01,577,26.144536,91.736172,247,car,average,5,0,...,1,0,False,True,False,False,True,False,False,3
8,8,BHMBCCMKT01,577,26.144536,91.736172,259,cycle,average,5,0,...,2,0,False,False,True,False,True,False,False,3
9,9,BHMBCCMKT01,577,26.144536,91.736172,266,bike,high,8,0,...,0,1,True,False,False,False,False,True,False,3
